<a href="https://colab.research.google.com/github/bkvkrll/Recommender-systems/blob/main/course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install implicit==0.4.4

# **Import libs**

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

# **Read data**

In [67]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# **Set global const**

In [68]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
N_PREDICT = 50 

# **Process features dataset**

In [69]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# **Split dataset for train, eval, test**

In [70]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [71]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [72]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [73]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [74]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (1831067, 12) Users: 2498 Items: 78118
val_matcher
Shape: (166081, 12) Users: 2132 Items: 27243
train_ranker
Shape: (166081, 12) Users: 2132 Items: 27243
val_ranker
Shape: (88634, 12) Users: 1955 Items: 21322


In [75]:
data_val_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
1828745,23,35486184136,524.0,1116369.0,1.0,1.99,319.0,0.0,15.0,76.0,0.0,0.0
1829314,164,35486258223,524.0,822970.0,1.0,3.79,404.0,0.0,3.0,76.0,0.0,0.0


# **Prefilter items**

In [76]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/content/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 78118 to 5001


# **Make cold-start to warm-start**

In [77]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (746228, 13) Users: 2495 Items: 5001
val_matcher
Shape: (166081, 12) Users: 2132 Items: 27243
train_ranker
Shape: (166081, 12) Users: 2132 Items: 27243
val_ranker
Shape: (88634, 12) Users: 1955 Items: 21322


# **Init/train recommender**

In [78]:
recommender = MainRecommender(data_train_matcher)

RuntimeError: ignored

# **Eval recall of matching**

In [79]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[888104.0, 913270.0, 995242.0, 1088462.0, 1124..."
1,2,"[825343.0, 826784.0, 833598.0, 858091.0, 88502..."


In [80]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=N_PREDICT))
result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=N_PREDICT))

NameError: ignored

In [81]:
%%time
# result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


In [82]:
# # сырой и простой пример как можно обернуть в функцию
def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()

In [83]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [84]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [85]:
TOPK_RECALL = 50

In [86]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[]

In [87]:
TOPK_PRECISION = 5

In [88]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[]

# **Ranking part**

In [89]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [90]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

NameError: ignored

In [91]:
df_match_candidates.head(2)

,user_id
0,23
1,164


In [92]:
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

KeyError: ignored

In [93]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

KeyError: ignored

In [95]:
df_match_candidates.head(4)

,user_id
0,23
1,164
2,278
3,718


In [96]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates


KeyError: ignored

In [97]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

df_ranker_train.head()

,user_id,item_id,target
1828745,23,1116369.0,1
1829314,164,822970.0,1
1829315,164,835098.0,1
1829316,164,839419.0,1
1829317,164,839605.0,1


In [98]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

KeyError: ignored

In [99]:
df_ranker_train.target.value_counts()

1    166081
Name: target, dtype: int64

In [100]:
df_ranker_train.head(2)

,user_id,item_id,target
1828745,23,1116369.0,1
1829314,164,822970.0,1


In [101]:
df_ranker_train['target'].mean()

1.0

# **Подготавливаем фичи для обучения модели**

In [102]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [103]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [104]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,23,1116369.0,1,1577,COSMETICS,National,MAKEUP AND TREATMENT,APPLICATORS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,164,822970.0,1,1083,GROCERY,National,HISPANIC,ORIENTAL FOODS AND MEALS,42 OZ,45-54,U,35-49K,Unknown,2 Adults No Kids,2,None/Unknown


In [105]:
df_join_train_matcher.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1.0,1004906.0,1.0,1.39,364.0,-0.60,1631.0,1.0,0.0,0.0
1,2375,26984851472,1.0,1033142.0,1.0,0.82,364.0,0.00,1631.0,1.0,0.0,0.0
2,2375,26984851472,1.0,1036325.0,1.0,0.99,364.0,-0.30,1631.0,1.0,0.0,0.0
3,2375,26984851472,1.0,1082185.0,1.0,1.21,364.0,0.00,1631.0,1.0,0.0,0.0
4,2375,26984851472,1.0,8160430.0,1.0,1.50,364.0,-0.39,1631.0,1.0,0.0,0.0


In [106]:
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)

In [107]:
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,total_quantity_value,item_freq,user_freq,total_user_sales_value,item_quantity_per_week,user_quantity_per_week,item_quantity_per_basket,user_quantity_per_baskter,item_freq_per_basket,user_freq_per_basket
0,23,1116369.0,1,1577,COSMETICS,National,MAKEUP AND TREATMENT,APPLICATORS,,NaN,...,20.0,19,624,2639.31,0.246914,2064.345679,0.000093,0.775818,0.000088,0.002895
1,164,822970.0,1,1083,GROCERY,National,HISPANIC,ORIENTAL FOODS AND MEALS,42 OZ,45-54,...,43.0,35,1328,3048.66,0.530864,746.740741,0.000200,0.280638,0.000162,0.006162
2,164,835098.0,1,69,PRODUCE,Private,POTATOES,POTATOES RED (BULK&BAG),5LB BAG,45-54,...,854.0,808,1328,3048.66,10.543210,746.740741,0.003962,0.280638,0.003749,0.006162
3,164,839419.0,1,69,MEAT-PCKGD,Private,MEAT - MISC,BREAST - BONELESS(IQF),3 LB,45-54,...,930.0,756,1328,3048.66,11.481481,746.740741,0.004315,0.280638,0.003508,0.006162
4,164,839605.0,1,2310,MEAT-PCKGD,National,FROZEN MEAT,BURRITOS,40 OZ,45-54,...,52.0,48,1328,3048.66,0.641975,746.740741,0.000241,0.280638,0.000223,0.006162


In [108]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [109]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [110]:
%%time
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=200,
                     learning_rate=0.1,
                     categorical_column=cat_feats,
                     n_jobs=-1)

lgb.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


CPU times: user 1.12 s, sys: 7.98 ms, total: 1.13 s
Wall time: 1.57 s


In [111]:
train_preds = lgb.predict_proba(X_train)

In [112]:
df_ranker_predict = df_ranker_train.copy()

In [113]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

# **Evaluation on test dataset**

In [114]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[840361.0, 856942.0, 857006.0, 859676.0, 86800..."
1,2,"[831125.0, 838136.0, 852864.0, 899624.0, 90864..."


In [115]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

NameError: ignored

In [116]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[]

# **Eval re-ranked matched result on test dataset**

In [117]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [118]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [119]:
# смотрим на метрики выше и сравниваем что с ранжированием и без, добавляем фичи и то же смотрим
# в первом приближении метрики должны расти с использованием второго этапа

print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.17021813592932963)


/content/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


# **Оценка на тесте для выполнения курсового проекта**

In [120]:
df_test = pd.read_csv('retail_test1.csv')

In [121]:
df_test.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [122]:
result_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [127]:
# weighting = [None, 'bm25', 'tfidf']
weighting = ['bm25', 'tfidf']
n_candidates = 50

In [128]:
%%time
for w in weighting:
    print(f'weighting: {w}...')
    recommender = MainRecommender(data_train_matcher, w)

    result_eval_matcher[f'own_rec {w}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=n_candidates))
    result_eval_matcher[f'sim_item_rec {w}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=n_candidates))
    result_eval_matcher[f'als_rec {w}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=n_candidates))
    result_eval_matcher[f'sim_user_rec {w}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=n_candidates))

weighting: bm25...


RuntimeError: ignored

In [129]:
TOP_K_PRECISION = 5

In [130]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [131]:
sorted(calc_precision(result_eval_matcher, TOP_K_PRECISION), key=lambda x: x[1],reverse=True)

[]

In [132]:
recommender = MainRecommender(data_train_matcher, 'bm25')
items_embendings, user_embedings = recommender.get_embeddings()
top_popular = recommender.get_top_popular()
candidates = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=n_candidates))

RuntimeError: ignored

In [133]:
items_emb_df = recommender.items_emb_df
users_emb_df = recommender.users_emb_df

NameError: ignored